In [2]:
import string
from collections import Counter

import requests as requests
import pandas as pd
from bs4 import BeautifulSoup

response = requests.get('https://minecraft.wiki/smelting')
print(response.status_code)
soup = BeautifulSoup(response.content, 'html.parser')

print(soup.p.get_text())

200
Smelting is the process of obtaining refined goods from raw materials by heating them in a furnace, blast furnace or smoker. When items are smelted in either type of furnace, experience is dropped. Like crafting, smelting uses recipes to determine what item is produced.



In [3]:
dfl = pd.read_html(response.content, header=0)
df = dfl[0]
dfstr = df.astype(str)
df

,Product,Ingredient,Exp,Usage
0,Baked Potato,Potato,0.35,"Fills 5 (), while raw fills 1 ()."
1,Dried Kelp,Kelp,0.10,Fills 1 () and can be eaten faster than other ...
2,Steak,Raw Beef,0.35,"Fills 8 (), while raw fills 3 ()."
3,Cooked Porkchop,Raw Porkchop,0.35,"Fills 8 (), while raw fills 3 ()."
4,Cooked Mutton,Raw Mutton,0.35,"Fills 6 (), while raw fills 2 ()."
5,Cooked Chicken,Raw Chicken,0.35,"Fills 6 (), while raw fills 2 () and has a 30%..."
6,Cooked Rabbit,Raw Rabbit,0.35,"Fills 5 (), while raw fills 3 ()."
7,Cooked Cod,Raw Cod,0.35,"Fills 5 (), while raw fills 2 ()."
8,Cooked Salmon,Raw Salmon,0.35,"Fills 6 (), while raw fills 2 ()."


In [4]:
words = (
    pd.Series(dfstr.values.flatten())
    .str.split()
    .explode()
    .value_counts()
    .reset_index()
)

words.columns = ['Word', 'Count']

words = words[words['Word'] != 'nan']
words


,Word,Count
0,Fills,9
1,while,9
2,raw,8
3,"(),",8
4,0.35,8
5,fills,8
6,().,7
7,Raw,7
8,Cooked,6
9,2,4


In [5]:
divs = soup.find('div', class_='mw-parser-output')
for edit_link in divs.find_all(class_='mw-editsection'):
    edit_link.decompose()

header_to_del = ['Navigation', 'References']

for header_name in header_to_del:
    header = divs.find(id=header_name)
    if header:
        header.decompose()

classes_to_del = ['mw-references-wrap']

for class_name in classes_to_del:
    classes = divs.find_all(class_=class_name)
    for clas in classes:
        clas.decompose()

for navbox in divs.find_all(class_='navbox'):
    navbox.decompose()

text = divs.get_text()

with open('smeltin_test.txt', 'w') as f:
    f.write(text)

# Some more classes to nuke found by llms
#     wrappers_to_nuke = [
#     'mw-references-wrap',
#     'navbox',
#     'infobox',
#     'toc',
#     'mw-editsection',
#     'hatnote',            # "See also:", "Main article:"
#     'searchaux',          # Klasa pomocnicza często występująca z hatnote/msgbox
#     'msgbox',             # Ramki z ostrzeżeniami/informacjami technicznymi
#     'thumb',              # Kontenery z obrazkami i podpisami (opcjonalnie)
#     'figure',             # Znacznik HTML5 dla obrazków (często na Wiki)
#     'catlinks',           # Kategorie na samym dole strony
#     'printfooter',        # "Retrieved from..." na dole
#     'mw-indicators'       # Ikonki w prawym górnym rogu (np. kłódka)
# ]



In [6]:
text = text.lower()

to_del_signs = string.punctuation + string.digits  + '×' + '–' + '⁄' + '\u200c'

for sign in to_del_signs:
    text = text.replace(sign, ' ')

words_list = text.split()

words_list

['the',
 'furnace',
 'interface',
 'note',
 'how',
 'the',
 'spaces',
 'are',
 'displayed',
 'smelting',
 'is',
 'the',
 'process',
 'of',
 'obtaining',
 'refined',
 'goods',
 'from',
 'raw',
 'materials',
 'by',
 'heating',
 'them',
 'in',
 'a',
 'furnace',
 'blast',
 'furnace',
 'or',
 'smoker',
 'when',
 'items',
 'are',
 'smelted',
 'in',
 'either',
 'type',
 'of',
 'furnace',
 'experience',
 'is',
 'dropped',
 'like',
 'crafting',
 'smelting',
 'uses',
 'recipes',
 'to',
 'determine',
 'what',
 'item',
 'is',
 'produced',
 'contents',
 'methods',
 'furnace',
 'blast',
 'furnace',
 'and',
 'smoker',
 'recipes',
 'food',
 'ores',
 'gear',
 'furnace',
 'only',
 'fuel',
 'hopper',
 'automation',
 'achievements',
 'advancements',
 'history',
 'java',
 'edition',
 'bedrock',
 'edition',
 'new',
 'nintendo',
 'ds',
 'edition',
 'issues',
 'trivia',
 'gallery',
 'screenshots',
 'see',
 'also',
 'references',
 'navigation',
 'methods',
 'furnace',
 'main',
 'article',
 'furnace',
 'the',
 

In [7]:
counter = Counter(words_list)
counter


Counter({'the': 159,
         'used': 121,
         'and': 98,
         'a': 90,
         'be': 77,
         'fuel': 76,
         'ticks': 73,
         'can': 72,
         'furnace': 66,
         'to': 65,
         'as': 61,
         'of': 56,
         'is': 50,
         'in': 47,
         'now': 43,
         'item': 39,
         'none': 36,
         'smelting': 33,
         'items': 33,
         'for': 30,
         'or': 28,
         'slot': 25,
         'edition': 24,
         'which': 24,
         'iron': 23,
         'block': 23,
         'smelted': 21,
         'only': 21,
         'wooden': 21,
         'w': 21,
         'raw': 20,
         'output': 20,
         'when': 19,
         'from': 18,
         'an': 18,
         'if': 18,
         'input': 18,
         'note': 17,
         'experience': 17,
         'fills': 17,
         'blocks': 17,
         'copper': 17,
         'with': 17,
         'also': 16,
         'smelt': 16,
         'added': 16,
         'furnaces': 16,
  

In [8]:
import json, os

if os.path.exists('word_counts.json'):
    with open('word_counts.json', 'r', encoding='utf-8') as f:
        old_json = json.load(f)
        old_counter = Counter(old_json)
else:
    old_counter = Counter()

old_counter.update(counter)

with open('word_counts.json', 'w', encoding='utf-8') as f:
    json.dump(counter, f, ensure_ascii=False, indent=4)



In [9]:
from wordfreq import word_frequency, top_n_list
word_frequency('the', 'en')
top_n_list('en', 10)

['the', 'to', 'and', 'of', 'a', 'in', 'i', 'is', 'for', 'that']

In [10]:

normalized_counter = {}

for k, v in counter.items():
    normalized_counter[k] = v / counter.total()

print(counter)
print(normalized_counter)

Counter({'the': 159, 'used': 121, 'and': 98, 'a': 90, 'be': 77, 'fuel': 76, 'ticks': 73, 'can': 72, 'furnace': 66, 'to': 65, 'as': 61, 'of': 56, 'is': 50, 'in': 47, 'now': 43, 'item': 39, 'none': 36, 'smelting': 33, 'items': 33, 'for': 30, 'or': 28, 'slot': 25, 'edition': 24, 'which': 24, 'iron': 23, 'block': 23, 'smelted': 21, 'only': 21, 'wooden': 21, 'w': 21, 'raw': 20, 'output': 20, 'when': 19, 'from': 18, 'an': 18, 'if': 18, 'input': 18, 'note': 17, 'experience': 17, 'fills': 17, 'blocks': 17, 'copper': 17, 'with': 17, 'also': 16, 'smelt': 16, 'added': 16, 'furnaces': 16, 'craft': 16, 'bricks': 15, 'overworld': 15, 'blast': 14, 'ore': 14, 'armor': 13, 'nether': 13, 'are': 12, 'recipes': 12, 'any': 12, 'while': 12, 'coal': 12, 'gold': 12, 'stone': 12, 'hopper': 11, 'java': 11, 'player': 11, 'not': 11, 'using': 11, 'all': 11, 'cooked': 11, 'various': 11, 'bamboo': 11, 'smoker': 10, 'crafting': 10, 'it': 10, 'smooth': 10, 'into': 9, 'more': 9, 'on': 9, 'has': 9, 'this': 9, 'tools': 9

In [11]:
df = pd.DataFrame.from_dict(normalized_counter, orient='index', columns=['Wiki Frequency'])

df['Language Frequency'] = df.index.map(lambda word: word_frequency(word, 'en'))


df


,Wiki Frequency,Language Frequency
the,0.043008,0.053700
furnace,0.017852,0.000004
interface,0.001352,0.000013
note,0.004598,0.000110
how,0.001352,0.001740
...,...,...
definition,0.000270,0.000038
encyclopedia,0.000270,0.000003
entry,0.000270,0.000047
you,0.000541,0.009550


In [12]:
df_top = pd.DataFrame(top_n_list('en', 5), columns=['Word'])
df_top.set_index('Word', inplace=True)
df_top['Language Frequency'] = df_top.index.map(lambda word: word_frequency(word, 'en'))

df_top

,Language Frequency
Word,
the,0.0537
to,0.0269
and,0.0257
of,0.0251
a,0.0229


In [21]:
url_prefix = "https://minecraft.wiki"
links = soup.find_all('a')

found_links = {
        url_prefix + link.get('href')
        for link in soup.find_all('a')
        if link.get('href', '').startswith('/w/')
    }

found_links


{'https://minecraft.wiki/w/Acacia_Boat',
 'https://minecraft.wiki/w/Acacia_Boat_with_Chest',
 'https://minecraft.wiki/w/Acacia_Button',
 'https://minecraft.wiki/w/Acacia_Door',
 'https://minecraft.wiki/w/Acacia_Fence',
 'https://minecraft.wiki/w/Acacia_Fence_Gate',
 'https://minecraft.wiki/w/Acacia_Hanging_Sign',
 'https://minecraft.wiki/w/Acacia_Leaves',
 'https://minecraft.wiki/w/Acacia_Log',
 'https://minecraft.wiki/w/Acacia_Planks',
 'https://minecraft.wiki/w/Acacia_Pressure_Plate',
 'https://minecraft.wiki/w/Acacia_Sapling',
 'https://minecraft.wiki/w/Acacia_Shelf',
 'https://minecraft.wiki/w/Acacia_Sign',
 'https://minecraft.wiki/w/Acacia_Slab',
 'https://minecraft.wiki/w/Acacia_Stairs',
 'https://minecraft.wiki/w/Acacia_Trapdoor',
 'https://minecraft.wiki/w/Acacia_Wood',
 'https://minecraft.wiki/w/Achievement#Acquire_Hardware',
 'https://minecraft.wiki/w/Achievement#Delicious_Fish',
 'https://minecraft.wiki/w/Achievement#Dry_Spell',
 'https://minecraft.wiki/w/Achievement#Pork_Ch